# 라이브러리 설정

In [3]:
#정보요청 라이브러리 import
import requests

#크롤링 라이브러리 import
from bs4 import BeautifulSoup 

#셀레늄 라이브러리 불러오기
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pandas as pd
import re
import time
import random

## URL 지정

In [2]:
#User-Agent 지정
headers =  # {'User-Agent' : '여기에 개인 User-Agent 지정할 것'}

In [4]:
# URL 설정 
# 임의로 미라클나잇으로 확인 
URL = 'https://play.google.com/store/apps/details?id=com.munice.miraclenight_android'

## 변수명 설정 

In [3]:
# id 
id_start = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div['
id_end = ']/header/div[1]/div[1]/div'

# date
date_start = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div['
date_end =']/header/div[2]/span'

# rating 
rating_start = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div['
rating_end =']/header/div[2]/div'

# review
review_start = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div['
review_end =']/div[1]'

# 유용
useful_start = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div['
useful_end =']/div[2]/div'

#리뷰 개수 
review_num = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[1]/div/div/div[1]/div[3]'

# 모든 리뷰 버튼 
all_review = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/header/div/div[2]/button'

## 드라이버 시작

In [ ]:
#드라이버 객체 생성
browser = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
browser.get(URL)

# 크롤링 개수 확인 및 설정

In [ ]:
# 크롤링할 리뷰 개수
review_count = browser.find_element(by = By.XPATH, value = review_num).text

ten_thousand = '만' in review_count
thousand = '천' in review_count

# 리뷰가 만개 이상인 경우 리뷰 개수 계산
if ten_thousand:
    start = review_count.find('뷰')
    end = review_count.find('만')
    
    review_count = (float(review_count[start+2:end])+0.01) * 10000

# 리뷰가 천개 이상인 경우 리뷰 개수 계산    
elif thousand:
    start = review_count.find('뷰')
    end = review_count.find('천')
    review_count = (float(review_count[start+2:end]) + 0.01)  * 1000

# else
else:
    start = review_count.find('뷰')
    end = review_count.find('개')
    review_count = review_count[start+2:end]

review_count = int(review_count)
review_count

In [20]:
# 그냥 디폴트 1000개로 설정할 때
# review_count = 1000

# 모든 리뷰 클릭 및 스크롤

In [153]:
browser.find_element(by = By.XPATH, value = all_review).click()    

In [4]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = browser.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            browser.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            browser.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = browser.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = browser.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
    except: 
        pass

In [161]:
modal = WebDriverWait(browser, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

스크롤 완료


<ipython-input-154-54060f592e59>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_review_button = browser.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()


# 리뷰 데이터 크롤

In [22]:
review_dict_total = {}

# rating만 따로 받아 옴
# html parsing하기
html_source = browser.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')
review_source = soup_source.find_all(class_ = 'RHo1pe')

# reating 데이터 저장
rating_list = []
for review in review_source:
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    rating_list.append(rating)

# 리뷰 데이터 전부 크롤링
for count in range(review_count):
    count+= 1
    try:
        review_dict = {}
        id_ = browser.find_element(by = By.XPATH, value = id_start + str(count) + id_end).text
        date_ = browser.find_element(by = By.XPATH, value = date_start + str(count) + date_end).text
        review_ = browser.find_element(by = By.XPATH, value = review_start + str(count) + review_end).text
    #    rating_ = browser.find_element(by = By.XPATH, value = rating_start + str(count) + rating_end).text
    #    useful_ = browser.find_element(by = By.XPATH, value = useful_start + str(count) + useful_end).text

        review_dict['id'] = id_
        review_dict['date'] = date_
        review_dict['rating'] = rating_list[count-1]
        review_dict['review'] = review_
    #    review_dict['useful'] = useful_

        review_dict_total['user_' + str(count)] = review_dict
    except:
        continue
                
review_dict_total    

{'user_1': {'id': '한새2',
  'date': '2022년 9월 9일',
  'rating': '5',
  'review': '(한달 사용 후기입니다) 이 앱을 켜두고 자면 진짜 편하게 잘 수 있습니다. 매번 9시간 넘게 자거나, 깊게 잠들지 못하고 수시로 깨는데, 이거 켜놨을 때 딱 4시간 반 자고, 알람을 맞추지 않아도 제때 일어날 수 있어서 좋았습니다. 엄마도 해가 서쪽에서 떴냐면서 깨우지 않아도 잘 일어난다며 참 좋아하십니다. 처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한데 이틀 지나면 적응이 끝난건지 괜찮아집니다. 그리고 가끔 아침에 일어났을 때 머리가 몽롱한데 그땐 아침밥을 먹거나 커피 한잔을 마셔주면 참 개운한 하루를 시작할 수 있습니다. (단점) 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌다 켜지면.. 그날은 지각하는거랍니다.. ㅜㅠ 앱에 표시되는 광고에 대한 평가란이 있었는데, 그런거 신경 안쓰고 잠잘때만 사용하다보니 앱에 광고가 있었다는걸 평가란을 보고 처음 알았습니다.'},
 'user_2': {'id': '김보경',
  'date': '2022년 8월 29일',
  'rating': '5',
  'review': '엄청 잘 잔거같은데 워치에는 3번이나 깨고 수면 점수 낮게 기록되네요 근데 또 꿈결에 윗집 진동알람땜에 짜증났었는데 앱 소리였는지 뭔지... 묘해요 일단 지금 엄청 상쾌하긴해요! 아직 베타 단계라 뭐 부족한 부분이 있다는 후기가 있어서 한번 경험하고 결제진행을 선택하긴 어렵네요 여튼 나쁘지 않은 경험이었어요'},
 'user_3': {'id': 'De Lune',
  'date': '2022년 7월 30일',
  'rating': '2',
  'review': '개인적인 문제일 수도 있지만 미라클나잇을 사용할 때마다 휴대폰 진동에 머리가 아파서 잠을 못자겠어요...😢 그리고 수면 중지를 해도 주파수 소리가 계속 나오고 알림창 알림을 들어가면 이용권이 없는데

# json 저장

In [165]:
import os
import json

file_path = os.getcwd()
file_name = '\\review_dict'

with open(file_path+file_name, 'w') as outfile:
    json.dump(review_dict_total, outfile, indent=4)

# json 읽기

In [166]:
import os
import json

file_path = os.getcwd()
file_name = '\\review_dict'

with open(file_path+file_name, "r") as json_file:
    json_data = json.load(json_file)
    print(json_data)
    print("")

{'user_1': {'id': '한새2', 'date': '2022년 9월 9일', 'rating': '', 'review': '(한달 사용 후기입니다) 이 앱을 켜두고 자면 진짜 편하게 잘 수 있습니다. 매번 9시간 넘게 자거나, 깊게 잠들지 못하고 수시로 깨는데, 이거 켜놨을 때 딱 4시간 반 자고, 알람을 맞추지 않아도 제때 일어날 수 있어서 좋았습니다. 엄마도 해가 서쪽에서 떴냐면서 깨우지 않아도 잘 일어난다며 참 좋아하십니다. 처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한데 이틀 지나면 적응이 끝난건지 괜찮아집니다. 그리고 가끔 아침에 일어났을 때 머리가 몽롱한데 그땐 아침밥을 먹거나 커피 한잔을 마셔주면 참 개운한 하루를 시작할 수 있습니다. (단점) 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌다 켜지면.. 그날은 지각하는거랍니다.. ㅜㅠ 앱에 표시되는 광고에 대한 평가란이 있었는데, 그런거 신경 안쓰고 잠잘때만 사용하다보니 앱에 광고가 있었다는걸 평가란을 보고 처음 알았습니다.'}, 'user_2': {'id': '김보경', 'date': '2022년 8월 29일', 'rating': '', 'review': '엄청 잘 잔거같은데 워치에는 3번이나 깨고 수면 점수 낮게 기록되네요 근데 또 꿈결에 윗집 진동알람땜에 짜증났었는데 앱 소리였는지 뭔지... 묘해요 일단 지금 엄청 상쾌하긴해요! 아직 베타 단계라 뭐 부족한 부분이 있다는 후기가 있어서 한번 경험하고 결제진행을 선택하긴 어렵네요 여튼 나쁘지 않은 경험이었어요'}, 'user_3': {'id': 'De Lune', 'date': '2022년 7월 30일', 'rating': '', 'review': '개인적인 문제일 수도 있지만 미라클나잇을 사용할 때마다 휴대폰 진동에 머리가 아파서 잠을 못자겠어요...😢 그리고 수면 중지를 해도 주파수 소리가 계속 나오고 알림창 알림을 들어가면 이용권이 없는데도 다시 수면이 시작되는 버그가 있습니다.

In [79]:
len(json_data.keys())

17

# 앱 이름으로 자동화

In [6]:
# 앱 리스트
Sleep_Application_dict = {
    '미라클나잇' : 'https://play.google.com/store/apps/details?id=com.munice.miraclenight_android',
    '코끼리'     : 'https://play.google.com/store/apps/details?id=com.mindclass.android',
    '수면소리'   : 'https://play.google.com/store/apps/details?id=sleepsounds.relaxandsleep.whitenoise',
    'Calm'      : 'https://play.google.com/store/apps/details?id=com.calm.android',
    '슬립사이클' : 'https://play.google.com/store/apps/details?id=com.northcube.sleepcycle',
    '수면분석'   : 'https://play.google.com/store/apps/details?id=sleeptrakcer.sleeprecorder.sleepapp.sleep',
    '스노어랩'   : 'https://play.google.com/store/apps/details?id=com.snorelab.app'
}

In [7]:
#드라이버 객체 생성
browser = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

In [ ]:
import os
import json
from tqdm.notebook import tqdm


for sleep_application in tqdm(list(Sleep_Application_dict.keys())):
    
    browser.get(Sleep_Application_dict[sleep_application])

    # 모든 리뷰 클릭 & 무한 스크롤
    browser.find_element(by = By.XPATH, value = all_review).click()
    modal = WebDriverWait(browser, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
    scroll(modal)


    # 리뷰 데이터 크롤 
    review_dict_total = {}

    # rating만 따로 받아 옴
    # html parsing하기
    html_source = browser.page_source
    soup_source = BeautifulSoup(html_source, 'html.parser')
    review_source = soup_source.find_all(class_ = 'RHo1pe')

    rating_list = []
    for review in review_source:
        rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
        rating_list.append(rating)
        
    # 가져올 리뷰 개수 지정 
    review_count = len(rating_list)

 
    for count in tqdm(range(review_count)):
        count+= 1
        try:
            review_dict = {}
            id_ = browser.find_element(by = By.XPATH, value = id_start + str(count) + id_end).text
            date_ = browser.find_element(by = By.XPATH, value = date_start + str(count) + date_end).text
            review_ = browser.find_element(by = By.XPATH, value = review_start + str(count) + review_end).text
        #    rating_ = browser.find_element(by = By.XPATH, value = rating_start + str(count) + rating_end).text
        #    useful_ = browser.find_element(by = By.XPATH, value = useful_start + str(count) + useful_end).text

            review_dict['id'] = id_
            review_dict['date'] = date_
            review_dict['rating'] = rating_list[count-1]
            review_dict['review'] = review_
        #    review_dict['useful'] = useful_

            review_dict_total['user_' + str(count)] = review_dict

            
            # 저장
            file_path = str(os.getcwd()) + '\\'
            file_name = str(sleep_application) + '_dict'

            with open(file_path+file_name, 'w') as outfile:
                json.dump(review_dict_total, outfile, indent=4)

        except: 
            continue

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-4-3e6a791da913>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_review_button = browser.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()



스크롤 완료


 14%|█▍        | 1/7 [00:06<00:41,  6.98s/it]

스크롤 완료



 29%|██▊       | 2/7 [08:50<13:29, 162.00s/it]